In [170]:
# import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import *
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [171]:
jd='''Role - Senior Big Data Engineer
Required Technical skills - Big Data, Python, Spark, Data brick, data Spark
Desired Experience Range - 5 + years
Location of Requirement - PAN India
Desired Competencies
Must-Have
Good hands on experience in Python programming
· Data Engineering experience using AWS core services (Lambda, Glue, EMR and RedShift)
· Required skill set- SQL, Airflow · Must – Have Experience with snowflake

· Responsibility – Will be accountable for build/test complex data pipelines (batch and near real time) · Expectation – Readable documentation of all the components being develop · Experience in writing SQLs and stored procedures · Working experience with RDBMS (Oracle / Teradata)
Good-to-Have
· Good understanding of Data warehouse
· Familiarize in ETL tools like Informatica
· Experience working with NoSQL database like DynamoDB or MongoDB .'''

In [172]:


def remove_stuff(jd):
    jd_clean = jd.replace("\xa0", "").replace("/", "").replace(".", ". ").replace("●", "")
    return jd_clean

In [173]:
def get_cv(): #cleaned, processed, nlped cv content
    url=r'C:\Users\Admin\Downloads\Job-Recomendation-main\Job-Recomendation-main\data\scrapped_data\UpdatedResumeDataSet.csv'
    return pd.read_csv(url) 
# db_expander = st.beta_expander(label='Submitted resume:')
# with db_expander:
df = get_cv()

# checking data

In [174]:
# cv data

In [175]:
df

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [176]:
df["All"]

KeyError: 'All'

In [ ]:
# Xem 5 dòng đầu
print(df.head())

# Kiểm tra tên cột
print("Columns:", df.columns.tolist())

# Kiểm tra giá trị null
print("Missing values:", df.isnull().sum())

   Unnamed: 0                                             degree  \
0           0                                                NaN   
1           1  ['Bachelor of Engineering,Computer Sciencenvis...   
2           2  ['Master of Engineering', 'Master in Computer ...   
3           3                                                NaN   
4           4  ['Bachelor of Engineering, Computer Science  n...   

                        email  \
0  Jangrasukanya066@gmail.com   
1     abhayownsthis@gmail.com   
2                         NaN   
3         abhi4navv@gmail.com   
4       monu.aniket@gmail.com   

                                           file_path mobile_number  \
0  cv_to_df/3bdfa685-56f1-4f83-866c-dbb20be51048.pdf           NaN   
1  cv_to_df/abhay_kumar_visualcv_resume_with_GTC.pdf       9978045   
2                     cv_to_df/Abhijit-Manepatil.pdf           NaN   
3  cv_to_df/Abhinav_Abhishek_visualcv_resume (1).pdf    8757656959   
4                             cv_to_df/Ani

In [ ]:
df["All"] = df["All"].str.replace("\r\n", " ")  # Thay thế xuống dòng
df["All"] = df["All"].str.replace("â¢", "-")   # Sửa lỗi encoding
df["All"] = df["All"].str.lower()
import string
df["All"] = df["All"].str.translate(str.maketrans("", "", string.punctuation))

In [ ]:
# job discription entered by recuriter

In [ ]:
jd

'Role - Senior Big Data Engineer\nRequired Technical skills - Big Data, Python, Spark, Data brick, data Spark\nDesired Experience Range - 5 + years\nLocation of Requirement - PAN India\nDesired Competencies\nMust-Have\nGood hands on experience in Python programming\n· Data Engineering experience using AWS core services (Lambda, Glue, EMR and RedShift)\n· Required skill set- SQL, Airflow · Must – Have Experience with snowflake\n\n· Responsibility – Will be accountable for build/test complex data pipelines (batch and near real time) · Expectation – Readable documentation of all the components being develop · Experience in writing SQLs and stored procedures · Working experience with RDBMS (Oracle / Teradata)\nGood-to-Have\n· Good understanding of Data warehouse\n· Familiarize in ETL tools like Informatica\n· Experience working with NoSQL database like DynamoDB or MongoDB .'

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re

def nlp(text: str):
    """
    Cải thiện xử lý văn bản:
    - Loại bỏ các ký tự không mong muốn.
    - Chuẩn hóa văn bản (lowercase, xóa stopwords, lemmatization).
    - Xử lý các lỗi encoding phổ biến.
    """
    # Xử lý lỗi encoding và ký tự đặc biệt
    text = text.replace("\n", " ").replace("\r", " ")
    text = re.sub(r"â¢", "-", text)  # Sửa lỗi encoding phổ biến
    text = re.sub(r"[^\w\s]", " ", text)  # Loại bỏ ký tự không mong muốn

    # Chuyển văn bản về chữ thường
    text = text.lower()

    # Tokenize văn bản
    word_sent = word_tokenize(text)

    # Loại bỏ stopwords và dấu câu
    _stopwords = set(stopwords.words('english') + list(punctuation) + ["●", "–", "’", "-", "``", "''"])
    word_sent = [word for word in word_sent if word not in _stopwords]

    # Lemmatization (chuẩn hóa từ về dạng gốc)
    lemmatizer = WordNetLemmatizer()
    clean_text = [lemmatizer.lemmatize(word) for word in word_sent]

    return clean_text

In [ ]:
NLP_Processed_JD=nlp(jd)


In [ ]:
NLP_Processed_JD

['role',
 'senior',
 'big',
 'data',
 'engineer',
 'required',
 'technical',
 'skill',
 'big',
 'data',
 'python',
 'spark',
 'data',
 'brick',
 'data',
 'spark',
 'desired',
 'experience',
 'range',
 '5',
 'year',
 'location',
 'requirement',
 'pan',
 'india',
 'desired',
 'competency',
 'must',
 'good',
 'hand',
 'experience',
 'python',
 'programming',
 'data',
 'engineering',
 'experience',
 'using',
 'aws',
 'core',
 'service',
 'lambda',
 'glue',
 'emr',
 'redshift',
 'required',
 'skill',
 'set',
 'sql',
 'airflow',
 'must',
 'experience',
 'snowflake',
 'responsibility',
 'accountable',
 'build',
 'test',
 'complex',
 'data',
 'pipeline',
 'batch',
 'near',
 'real',
 'time',
 'expectation',
 'readable',
 'documentation',
 'component',
 'develop',
 'experience',
 'writing',
 'sqls',
 'stored',
 'procedure',
 'working',
 'experience',
 'rdbms',
 'oracle',
 'teradata',
 'good',
 'good',
 'understanding',
 'data',
 'warehouse',
 'familiarize',
 'etl',
 'tool',
 'like',
 'informatic

In [ ]:
################################################################################################################################

In [ ]:
jd_df=pd.DataFrame()
# jd_df['hi']=['I']
jd_df['jd']=[' '.join(NLP_Processed_JD)]

In [ ]:
jd_df['jd'].values[0]

'role senior big data engineer required technical skill big data python spark data brick data spark desired experience range 5 year location requirement pan india desired competency must good hand experience python programming data engineering experience using aws core service lambda glue emr redshift required skill set sql airflow must experience snowflake responsibility accountable build test complex data pipeline batch near real time expectation readable documentation component develop experience writing sqls stored procedure working experience rdbms oracle teradata good good understanding data warehouse familiarize etl tool like informatica experience working nosql database like dynamodb mongodb'

In [ ]:
clean_jd=jd_df['jd'].values[0]

In [ ]:
##############################################################################################################################

In [ ]:
df["All"]

0     sukanya jangrabhiwani haryanajangrasukanya066g...
1     abhay kumarlead data scientist  computer visio...
2     abhijit manepatildata scientist with master in...
3     abhinav abhisheksenior software developerdata ...
4     aniket kashyapsoftware developerdata scientist...
5     anirudh anandmachine learning engineeranirudha...
6     ashray rainaashrayraina22gmailcom 7780995937 a...
7     ramachandran iyeriim ahmedabad alumnus smp  cs...
8     ramachandran iyeriim ahmedabad alumnus smp  cs...
9     himanshu tyagipune maharashtra  email me on in...
10    jalpa davedata scientist  elegant microweb tec...
11    kishore ramiyengar lakshmipprasannac 404 gaana...
12    mohnish chaudharydecision scientist  mu sigmab...
13    pawan kumar apawan9964gmailcom45 8th cross roa...
14    prikshit sharmajammuindia prikshitsharma8024gm...
15    priyanka srivastavadata scientistdata science ...
16    rakshit khajuriamachine learning engineertalab...
17    rakshit khajuriamachine learning engineera

In [ ]:
cv_data=[]
for i in range(len(df["All"])):
    NLP_Processed_cv=nlp(df["All"].values[i])
    cv_data.append(NLP_Processed_cv)

In [ ]:
cv_data

[['sukanya',
  'jangrabhiwani',
  'haryanajangrasukanya066gmailcompersonal',
  'detailsdate',
  'birth',
  '20031124eligible',
  'work',
  'indiahighest',
  'career',
  'level',
  'fresherindustry',
  'internet',
  'ecommercework',
  'experiencecomputer',
  'operatoreducationbachelorsskills',
  'skill',
  'computer',
  'skill',
  'data',
  'entry',
  'typing',
  'customer',
  'service',
  'microsoft',
  'word',
  'microsoft',
  'excel',
  'microsoft',
  'office',
  'html',
  'javascript',
  'cccertifications',
  'licensesdiploma',
  'computer',
  'applicationjanuary',
  '2021',
  'april',
  '2022'],
 ['abhay',
  'kumarlead',
  'data',
  'scientist',
  'computer',
  'visionbangalore919731812812',
  'abhayownsthisgmailcom',
  'inlinkedincominabhaykumar99780458',
  'abhaymisegithubiosummary',
  'skillsi',
  'applied',
  'computer',
  'vision',
  'data',
  'scientist',
  'experience',
  'leading',
  'bunch',
  'core',
  'pythonother',
  'data',
  'scientist',
  'used',
  'leadership',
  'a

In [ ]:
cv_=[]
for i in cv_data:
    cv_.append([' '.join(i)])

In [ ]:
cv_

[['sukanya jangrabhiwani haryanajangrasukanya066gmailcompersonal detailsdate birth 20031124eligible work indiahighest career level fresherindustry internet ecommercework experiencecomputer operatoreducationbachelorsskills skill computer skill data entry typing customer service microsoft word microsoft excel microsoft office html javascript cccertifications licensesdiploma computer applicationjanuary 2021 april 2022'],
 ['abhay kumarlead data scientist computer visionbangalore919731812812 abhayownsthisgmailcom inlinkedincominabhaykumar99780458 abhaymisegithubiosummary skillsi applied computer vision data scientist experience leading bunch core pythonother data scientist used leadership applied deep learning skill successfullyexecute various computer vision project highly skilled applied deep learning computervision good understanding various computer vision technique like classification boundingbox classification regressionobject detectionpixel wise predictionsegmentation numpysequence 

In [ ]:
df["clean_all"]=pd.DataFrame(cv_)

# Recomendation Function

In [ ]:
df.head(1)

,Unnamed: 0,degree,email,file_path,mobile_number,name,no_of_pages,skills,All,pdf_to_base64,clean_all
0,0,NaN,Jangrasukanya066@gmail.com,cv_to_df/3bdfa685-56f1-4f83-866c-dbb20be51048.pdf,NaN,Sukanya Jangra,1.0,"['Html', 'C++', 'Javascript', 'Excel', 'Word',...",sukanya jangrabhiwani haryanajangrasukanya066g...,JVBERi0xLjQKJaqrrK0KMSAwIG9iago8PAovVGl0bGUgKE...,sukanya jangrabhiwani haryanajangrasukanya066g...


In [ ]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['name', 'degree',"email",'Unnamed: 0','mobile_number','skills','no_of_pages','score'])
    count = 0
    for i in top:
#         recommendation.at[count, 'ApplicantID'] = u
        
        recommendation.at[count, 'name'] = df['name'][i]
        recommendation.at[count, 'degree'] = df['degree'][i]
        recommendation.at[count, 'email'] = df['email'][i]
        recommendation.at[count, 'Unnamed: 0'] = df.index[i]
        recommendation.at[count, 'mobile_number'] = df['mobile_number'][i]
        recommendation.at[count, 'skills'] = df['skills'][i]
        recommendation.at[count, 'no_of_pages'] = df['no_of_pages'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

# cosine_similarity + TfidfVectorizer function

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


def TFIDF(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # TF-IDF Scraped data
    tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data)

    # TF-IDF CV
    user_tfidf = tfidf_vectorizer.transform(cv)

    # Using cosine_similarity on (Scraped data) & (CV)
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_jobid)

    output2 = list(cos_similarity_tfidf)
    return output2
    


In [ ]:
output2 = TFIDF(df["clean_all"],jd_df['jd'])

In [ ]:
output2 

[array([[0.03996723]]),
 array([[0.11737128]]),
 array([[0.15287517]]),
 array([[0.12705638]]),
 array([[0.13940233]]),
 array([[0.06631543]]),
 array([[0.06672218]]),
 array([[0.09318307]]),
 array([[0.09318307]]),
 array([[0.15121124]]),
 array([[0.13621604]]),
 array([[0.18148596]]),
 array([[0.06038984]]),
 array([[0.06689596]]),
 array([[0.08477842]]),
 array([[0.06241054]]),
 array([[0.09498649]]),
 array([[0.10645813]]),
 array([[0.04617904]])]

In [ ]:
print("👉 Các cột trong df:", df.columns.tolist())

👉 Các cột trong df: ['Unnamed: 0', 'degree', 'email', 'file_path', 'mobile_number', 'name', 'no_of_pages', 'skills', 'All', 'pdf_to_base64', 'clean_all']


In [ ]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:100]
list_scores = [output2[i][0][0] for i in top]
TF=get_recommendation(top,df, list_scores)
TF

,name,degree,email,Unnamed: 0,mobile_number,skills,no_of_pages,score
0,Kishore Ramiyengar,['Bachelor of Technology (Civil)'],kishor.priya@gmail.com,11,007-2008,"['Conversion', 'Algorithms', 'Database', 'Heal...",9.0,0.181486
1,Abhijit Manepatil,"['Master of Engineering', 'Master in Computer ...",NaN,2,NaN,"['Modeling', 'English', 'Gis', 'Website', 'Alg...",3.0,0.152875
2,Himanshu Tyagi,['BTech in Electronics'],NaN,9,0800189689,"['Presentation', 'Queries', 'Pattern', 'Datase...",2.0,0.151211
3,Aniket Kashyap,"['Bachelor of Engineering, Computer Science n...",monu.aniket@gmail.com,4,9691316944,"['Website', 'Algorithms', 'Database', 'Visual'...",3.0,0.139402
4,Jalpa Dave,"['M.Tech', 'B.E. in Computer']",NaN,10,NaN,"['Website', 'Brand', 'Algorithms', 'Database',...",3.0,0.136216
5,Abhinav Abhishek,NaN,abhi4navv@gmail.com,3,8757656959,"['Warehouse', 'Training', 'Networking', 'Datab...",3.0,0.127056
6,Abhay Kumar,"['Bachelor of Engineering,Computer Sciencenvis...",abhayownsthis@gmail.com,1,9978045,"['Caffe', 'Modeling', 'Website', 'Training', '...",4.0,0.117371
7,Rakshit Khajuria,['Electronics and Communication nEngineering S...,rakshitraina1234@gmail.com,17,9149556155,"['Modeling', 'Website', 'Aws', 'Design', 'Stat...",1.0,0.106458
8,Rakshit Khajuria,"['Electronics and Communication, Engineering S...",rakshitraina1234@gmail.com,16,9149556155,"['Modeling', 'Acquisition', 'Presentation', 'A...",2.0,0.094986
9,Ramachandran Iyer,['Bachelors of Commerce'],Ganesh2480@gmail.com,7,9820463399,"['Website', 'Queries', 'Database', 'Design', '...",2.0,0.093183


In [ ]:
df

,Unnamed: 0,degree,email,file_path,mobile_number,name,no_of_pages,skills,All,pdf_to_base64,clean_all
0,0,NaN,Jangrasukanya066@gmail.com,cv_to_df/3bdfa685-56f1-4f83-866c-dbb20be51048.pdf,NaN,Sukanya Jangra,1.0,"['Html', 'C++', 'Javascript', 'Excel', 'Word',...",sukanya jangrabhiwani haryanajangrasukanya066g...,JVBERi0xLjQKJaqrrK0KMSAwIG9iago8PAovVGl0bGUgKE...,sukanya jangrabhiwani haryanajangrasukanya066g...
1,1,"['Bachelor of Engineering,Computer Sciencenvis...",abhayownsthis@gmail.com,cv_to_df/abhay_kumar_visualcv_resume_with_GTC.pdf,9978045,Abhay Kumar,4.0,"['Caffe', 'Modeling', 'Website', 'Training', '...",abhay kumarlead data scientist computer visio...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAYg...,abhay kumarlead data scientist computer vision...
2,2,"['Master of Engineering', 'Master in Computer ...",NaN,cv_to_df/Abhijit-Manepatil.pdf,NaN,Abhijit Manepatil,3.0,"['Modeling', 'English', 'Gis', 'Website', 'Alg...",abhijit manepatildata scientist with master in...,JVBERi0xLjQKJaqrrK0KNCAwIG9iago8PAovVGl0bGUgKE...,abhijit manepatildata scientist master compute...
3,3,NaN,abhi4navv@gmail.com,cv_to_df/Abhinav_Abhishek_visualcv_resume (1).pdf,8757656959,Abhinav Abhishek,3.0,"['Warehouse', 'Training', 'Networking', 'Datab...",abhinav abhisheksenior software developerdata ...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAYg...,abhinav abhisheksenior software developerdata ...
4,4,"['Bachelor of Engineering, Computer Science n...",monu.aniket@gmail.com,cv_to_df/Aniket_cv.pdf,9691316944,Aniket Kashyap,3.0,"['Website', 'Algorithms', 'Database', 'Visual'...",aniket kashyapsoftware developerdata scientist...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAbg...,aniket kashyapsoftware developerdata scientist...
5,5,NaN,anirudhanand80@gmail.com,cv_to_df/Anirudh_Anand_Resume.pdf,8082366408,Anirudh Anand,2.0,"['Seaborn', 'Pandas', 'Matplotlib', 'Analysis'...",anirudh anandmachine learning engineeranirudha...,JVBERi0xLjQKJdPr6eEKMSAwIG9iago8PC9DcmVhdG9yIC...,anirudh anandmachine learning engineeranirudha...
6,6,['B.tech'],ashrayraina22@gmail.com,cv_to_df/ASHRAY_RAINA_Resume_13-02-2023-19-28-...,7780995937,ASHRAY RAINA,1.0,"['Statistical analysis', 'Seaborn', 'Matplotli...",ashray rainaashrayraina22gmailcom 7780995937 a...,JVBERi0xLjQKJdPr6eEKMSAwIG9iago8PC9DcmVhdG9yIC...,ashray rainaashrayraina22gmailcom 7780995937 a...
7,7,['Bachelors of Commerce'],Ganesh2480@gmail.com,cv_to_df/fe0e9156-9600-4679-a1cc-77cd4a2c122a.pdf,9820463399,Ramachandran Iyer,2.0,"['Website', 'Queries', 'Database', 'Design', '...",ramachandran iyeriim ahmedabad alumnus smp cs...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...,ramachandran iyeriim ahmedabad alumnus smp csm...
8,8,['Bachelors of Commerce'],Ganesh2480@gmail.com,cv_to_df/fe826f24-8900-4317-959f-cb4689540997.pdf,9820463399,Ramachandran Iyer,2.0,"['Website', 'Queries', 'Database', 'Design', '...",ramachandran iyeriim ahmedabad alumnus smp cs...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...,ramachandran iyeriim ahmedabad alumnus smp csm...
9,9,['BTech in Electronics'],NaN,cv_to_df/Himanshu-Tyagi.pdf,0800189689,Himanshu Tyagi,2.0,"['Presentation', 'Queries', 'Pattern', 'Datase...",himanshu tyagipune maharashtra email me on in...,JVBERi0xLjQKJaqrrK0KNCAwIG9iago8PAovVGl0bGUgKE...,himanshu tyagipune maharashtra email indeed in...


# cosine_similarity + CountVectorizer


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer()
count_jobid = count_vectorizer.fit_transform(df["clean_all"]) #converting job data into vectors using count vectorizers
count_jobid

<19x3047 sparse matrix of type '<class 'numpy.int64'>'
	with 5166 stored elements in Compressed Sparse Row format>

In [ ]:
user_count = count_vectorizer.transform(jd_df['jd'])#converting user cv data into vectors using count vectorizers
user_count

<1x3047 sparse matrix of type '<class 'numpy.int64'>'
	with 49 stored elements in Compressed Sparse Row format>

In [ ]:
print(user_count)

  (0, 434)	1
  (0, 470)	1
  (0, 490)	2
  (0, 527)	1
  (0, 685)	1
  (0, 691)	1
  (0, 751)	1
  (0, 814)	7
  (0, 820)	1
  (0, 909)	1
  (0, 953)	1
  (0, 1027)	1
  (0, 1030)	1
  (0, 1084)	7
  (0, 1263)	3
  (0, 1409)	1
  (0, 1637)	2
  (0, 1665)	1
  (0, 1843)	1
  (0, 1861)	2
  (0, 1885)	1
  (0, 1923)	1
  (0, 1999)	1
  (0, 2087)	1
  (0, 2189)	1
  (0, 2220)	1
  (0, 2260)	2
  (0, 2289)	1
  (0, 2300)	1
  (0, 2359)	2
  (0, 2360)	1
  (0, 2373)	1
  (0, 2398)	1
  (0, 2494)	1
  (0, 2497)	1
  (0, 2528)	2
  (0, 2574)	2
  (0, 2584)	1
  (0, 2613)	1
  (0, 2719)	1
  (0, 2742)	1
  (0, 2786)	1
  (0, 2805)	1
  (0, 2874)	1
  (0, 2909)	1
  (0, 2976)	1
  (0, 3020)	2
  (0, 3025)	1
  (0, 3033)	1


In [ ]:
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)
output3 = list(cos_similarity_countv)
#output3

In [ ]:
output3 

[array([[0.13018891]]),
 array([[0.26638094]]),
 array([[0.35363006]]),
 array([[0.25098925]]),
 array([[0.25045458]]),
 array([[0.21542344]]),
 array([[0.20382505]]),
 array([[0.1509526]]),
 array([[0.1509526]]),
 array([[0.33310629]]),
 array([[0.18371791]]),
 array([[0.3368092]]),
 array([[0.19737862]]),
 array([[0.17103859]]),
 array([[0.19682713]]),
 array([[0.22437473]]),
 array([[0.19819782]]),
 array([[0.22164832]]),
 array([[0.13473378]])]

In [ ]:
top = sorted(range(len(output3)), key=lambda i: output3[i], reverse=True)[:100]
list_scores = [output3[i][0][0] for i in top]
cv=get_recommendation(top, df, list_scores)
cv

,name,degree,email,Unnamed: 0,mobile_number,skills,no_of_pages,score
0,Abhijit Manepatil,"['Master of Engineering', 'Master in Computer ...",NaN,2,NaN,"['Modeling', 'English', 'Gis', 'Website', 'Alg...",3.0,0.35363
1,Kishore Ramiyengar,['Bachelor of Technology (Civil)'],kishor.priya@gmail.com,11,007-2008,"['Conversion', 'Algorithms', 'Database', 'Heal...",9.0,0.336809
2,Himanshu Tyagi,['BTech in Electronics'],NaN,9,0800189689,"['Presentation', 'Queries', 'Pattern', 'Datase...",2.0,0.333106
3,Abhay Kumar,"['Bachelor of Engineering,Computer Sciencenvis...",abhayownsthis@gmail.com,1,9978045,"['Caffe', 'Modeling', 'Website', 'Training', '...",4.0,0.266381
4,Abhinav Abhishek,NaN,abhi4navv@gmail.com,3,8757656959,"['Warehouse', 'Training', 'Networking', 'Datab...",3.0,0.250989
5,Aniket Kashyap,"['Bachelor of Engineering, Computer Science n...",monu.aniket@gmail.com,4,9691316944,"['Website', 'Algorithms', 'Database', 'Visual'...",3.0,0.250455
6,Priyanka Srivastava,"['B.Tech in Information Technology', 'PG certi...",priyankasrvstv16@gmail.com,15,8948669980,"['Parser', 'English', 'Pandas', 'Json', 'Pyspa...",1.0,0.224375
7,Rakshit Khajuria,['Electronics and Communication nEngineering S...,rakshitraina1234@gmail.com,17,9149556155,"['Modeling', 'Website', 'Aws', 'Design', 'Stat...",1.0,0.221648
8,Anirudh Anand,NaN,anirudhanand80@gmail.com,5,8082366408,"['Seaborn', 'Pandas', 'Matplotlib', 'Analysis'...",2.0,0.215423
9,ASHRAY RAINA,['B.tech'],ashrayraina22@gmail.com,6,7780995937,"['Statistical analysis', 'Seaborn', 'Matplotli...",1.0,0.203825


# TfidfVectorizer + NearestNeighbors

In [ ]:
from sklearn.neighbors import NearestNeighbors

def KNN(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    
    KNN = NearestNeighbors(n_neighbors=7,p=2)
    KNN.fit(tfidf_vectorizer.fit_transform(scraped_data))
#     NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv), return_distance=True)
    NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv))
    top = NNs[1][0][1:]
    index_score = NNs[0][0][1:]

    knn = get_recommendation(top, df, index_score)
    return knn

knn = KNN(df['clean_all'], jd_df['jd'])
knn

,name,degree,email,Unnamed: 0,mobile_number,skills,no_of_pages,score
0,Abhijit Manepatil,"['Master of Engineering', 'Master in Computer ...",NaN,2,NaN,"['Modeling', 'English', 'Gis', 'Website', 'Alg...",3.0,1.301633
1,Himanshu Tyagi,['BTech in Electronics'],NaN,9,0800189689,"['Presentation', 'Queries', 'Pattern', 'Datase...",2.0,1.302911
2,Aniket Kashyap,"['Bachelor of Engineering, Computer Science n...",monu.aniket@gmail.com,4,9691316944,"['Website', 'Algorithms', 'Database', 'Visual'...",3.0,1.311943
3,Jalpa Dave,"['M.Tech', 'B.E. in Computer']",NaN,10,NaN,"['Website', 'Brand', 'Algorithms', 'Database',...",3.0,1.31437
4,Abhinav Abhishek,NaN,abhi4navv@gmail.com,3,8757656959,"['Warehouse', 'Training', 'Networking', 'Datab...",3.0,1.32132
5,Abhay Kumar,"['Bachelor of Engineering,Computer Sciencenvis...",abhayownsthis@gmail.com,1,9978045,"['Caffe', 'Modeling', 'Website', 'Training', '...",4.0,1.32863


In [ ]:

knn.to_csv('knn_cv.csv', index=False)

# Combined TFIDF, CV and KNN result together, to make a dataframe "final"

In [ ]:
knn.columns

Index(['name', 'degree', 'email', 'Unnamed: 0', 'mobile_number', 'skills',
       'no_of_pages', 'score'],
      dtype='object')

In [ ]:
merge1 = knn[['Unnamed: 0','name', 'score']].merge(TF[['Unnamed: 0','score']], on= "Unnamed: 0")
final = merge1.merge(cv[['Unnamed: 0','score']], on = 'Unnamed: 0')
final = final.rename(columns={"score_x": "KNN", "score_y": "TF-IDF","score": "CV"})
final.head()

,Unnamed: 0,name,KNN,TF-IDF,CV
0,2,Abhijit Manepatil,1.301633,0.152875,0.35363
1,9,Himanshu Tyagi,1.302911,0.151211,0.333106
2,4,Aniket Kashyap,1.311943,0.139402,0.250455
3,10,Jalpa Dave,1.31437,0.136216,0.183718
4,3,Abhinav Abhishek,1.32132,0.127056,0.250989


In [ ]:
final

,Unnamed: 0,name,KNN,TF-IDF,CV
0,2,Abhijit Manepatil,1.301633,0.152875,0.35363
1,9,Himanshu Tyagi,1.302911,0.151211,0.333106
2,4,Aniket Kashyap,1.311943,0.139402,0.250455
3,10,Jalpa Dave,1.31437,0.136216,0.183718
4,3,Abhinav Abhishek,1.32132,0.127056,0.250989
5,1,Abhay Kumar,1.32863,0.117371,0.266381


In [ ]:
# Scale it
from sklearn.preprocessing import MinMaxScaler
slr = MinMaxScaler()
final[["KNN", "TF-IDF", 'CV']] = slr.fit_transform(final[["KNN", "TF-IDF", 'CV']])

# Multiply by weights
final['KNN'] = (1-final['KNN'])/3
final['TF-IDF'] = final['TF-IDF']/3
final['CV'] = final['CV']/3
final['Final'] = final['KNN']+final['TF-IDF']+final['CV']
final.sort_values(by="Final", ascending=False)

,Unnamed: 0,name,KNN,TF-IDF,CV,Final
0,2,Abhijit Manepatil,0.333333,0.333333,0.333333,1.000000
1,9,Himanshu Tyagi,0.317557,0.317711,0.293070,0.928338
2,4,Aniket Kashyap,0.206034,0.206842,0.130924,0.543800
3,10,Jalpa Dave,0.176074,0.176927,0.000000,0.353001
4,3,Abhinav Abhishek,0.090254,0.090930,0.131973,0.313157
5,1,Abhay Kumar,0.000000,0.000000,0.162168,0.162168


In [ ]:
# final.to_csv('final.csv', index=False)
final2 = final.sort_values(by="Final", ascending=False).copy()
final2

,Unnamed: 0,name,KNN,TF-IDF,CV,Final
0,2,Abhijit Manepatil,0.333333,0.333333,0.333333,1.000000
1,9,Himanshu Tyagi,0.317557,0.317711,0.293070,0.928338
2,4,Aniket Kashyap,0.206034,0.206842,0.130924,0.543800
3,10,Jalpa Dave,0.176074,0.176927,0.000000,0.353001
4,3,Abhinav Abhishek,0.090254,0.090930,0.131973,0.313157
5,1,Abhay Kumar,0.000000,0.000000,0.162168,0.162168


In [ ]:
# final2.merge(df, on="JobID")
final_df = df.merge(final2, on='Unnamed: 0')
final_df

,Unnamed: 0,degree,email,file_path,mobile_number,name_x,no_of_pages,skills,All,pdf_to_base64,clean_all,name_y,KNN,TF-IDF,CV,Final
0,1,"['Bachelor of Engineering,Computer Sciencenvis...",abhayownsthis@gmail.com,cv_to_df/abhay_kumar_visualcv_resume_with_GTC.pdf,9978045,Abhay Kumar,4.0,"['Caffe', 'Modeling', 'Website', 'Training', '...",abhay kumarlead data scientist computer visio...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAYg...,abhay kumarlead data scientist computer vision...,Abhay Kumar,0.000000,0.000000,0.162168,0.162168
1,2,"['Master of Engineering', 'Master in Computer ...",NaN,cv_to_df/Abhijit-Manepatil.pdf,NaN,Abhijit Manepatil,3.0,"['Modeling', 'English', 'Gis', 'Website', 'Alg...",abhijit manepatildata scientist with master in...,JVBERi0xLjQKJaqrrK0KNCAwIG9iago8PAovVGl0bGUgKE...,abhijit manepatildata scientist master compute...,Abhijit Manepatil,0.333333,0.333333,0.333333,1.000000
2,3,NaN,abhi4navv@gmail.com,cv_to_df/Abhinav_Abhishek_visualcv_resume (1).pdf,8757656959,Abhinav Abhishek,3.0,"['Warehouse', 'Training', 'Networking', 'Datab...",abhinav abhisheksenior software developerdata ...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAYg...,abhinav abhisheksenior software developerdata ...,Abhinav Abhishek,0.090254,0.090930,0.131973,0.313157
3,4,"['Bachelor of Engineering, Computer Science n...",monu.aniket@gmail.com,cv_to_df/Aniket_cv.pdf,9691316944,Aniket Kashyap,3.0,"['Website', 'Algorithms', 'Database', 'Visual'...",aniket kashyapsoftware developerdata scientist...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAbg...,aniket kashyapsoftware developerdata scientist...,Aniket Kashyap,0.206034,0.206842,0.130924,0.543800
4,9,['BTech in Electronics'],NaN,cv_to_df/Himanshu-Tyagi.pdf,0800189689,Himanshu Tyagi,2.0,"['Presentation', 'Queries', 'Pattern', 'Datase...",himanshu tyagipune maharashtra email me on in...,JVBERi0xLjQKJaqrrK0KNCAwIG9iago8PAovVGl0bGUgKE...,himanshu tyagipune maharashtra email indeed in...,Himanshu Tyagi,0.317557,0.317711,0.293070,0.928338
5,10,"['M.Tech', 'B.E. in Computer']",NaN,cv_to_df/Jalpa-Dave.pdf,NaN,Jalpa Dave,3.0,"['Website', 'Brand', 'Algorithms', 'Database',...",jalpa davedata scientist elegant microweb tec...,JVBERi0xLjQKJaqrrK0KNCAwIG9iago8PAovVGl0bGUgKE...,jalpa davedata scientist elegant microweb tech...,Jalpa Dave,0.176074,0.176927,0.000000,0.353001


In [ ]:
df.columns

Index(['Unnamed: 0', 'degree', 'email', 'file_path', 'mobile_number', 'name',
       'no_of_pages', 'skills', 'All', 'pdf_to_base64', 'clean_all'],
      dtype='object')

In [ ]:
final_df.columns

Index(['Unnamed: 0', 'degree', 'email', 'file_path', 'mobile_number', 'name_x',
       'no_of_pages', 'skills', 'All', 'pdf_to_base64', 'clean_all', 'name_y',
       'KNN', 'TF-IDF', 'CV', 'Final'],
      dtype='object')

In [ ]:
final_df

,Unnamed: 0,degree,email,file_path,mobile_number,name_x,no_of_pages,skills,All,pdf_to_base64,clean_all,name_y,KNN,TF-IDF,CV,Final
0,1,"['Bachelor of Engineering,Computer Sciencenvis...",abhayownsthis@gmail.com,cv_to_df/abhay_kumar_visualcv_resume_with_GTC.pdf,9978045,Abhay Kumar,4.0,"['Caffe', 'Modeling', 'Website', 'Training', '...",abhay kumarlead data scientist computer visio...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAYg...,abhay kumarlead data scientist computer vision...,Abhay Kumar,0.000000,0.000000,0.162168,0.162168
1,2,"['Master of Engineering', 'Master in Computer ...",NaN,cv_to_df/Abhijit-Manepatil.pdf,NaN,Abhijit Manepatil,3.0,"['Modeling', 'English', 'Gis', 'Website', 'Alg...",abhijit manepatildata scientist with master in...,JVBERi0xLjQKJaqrrK0KNCAwIG9iago8PAovVGl0bGUgKE...,abhijit manepatildata scientist master compute...,Abhijit Manepatil,0.333333,0.333333,0.333333,1.000000
2,3,NaN,abhi4navv@gmail.com,cv_to_df/Abhinav_Abhishek_visualcv_resume (1).pdf,8757656959,Abhinav Abhishek,3.0,"['Warehouse', 'Training', 'Networking', 'Datab...",abhinav abhisheksenior software developerdata ...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAYg...,abhinav abhisheksenior software developerdata ...,Abhinav Abhishek,0.090254,0.090930,0.131973,0.313157
3,4,"['Bachelor of Engineering, Computer Science n...",monu.aniket@gmail.com,cv_to_df/Aniket_cv.pdf,9691316944,Aniket Kashyap,3.0,"['Website', 'Algorithms', 'Database', 'Visual'...",aniket kashyapsoftware developerdata scientist...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/AEEAbg...,aniket kashyapsoftware developerdata scientist...,Aniket Kashyap,0.206034,0.206842,0.130924,0.543800
4,9,['BTech in Electronics'],NaN,cv_to_df/Himanshu-Tyagi.pdf,0800189689,Himanshu Tyagi,2.0,"['Presentation', 'Queries', 'Pattern', 'Datase...",himanshu tyagipune maharashtra email me on in...,JVBERi0xLjQKJaqrrK0KNCAwIG9iago8PAovVGl0bGUgKE...,himanshu tyagipune maharashtra email indeed in...,Himanshu Tyagi,0.317557,0.317711,0.293070,0.928338
5,10,"['M.Tech', 'B.E. in Computer']",NaN,cv_to_df/Jalpa-Dave.pdf,NaN,Jalpa Dave,3.0,"['Website', 'Brand', 'Algorithms', 'Database',...",jalpa davedata scientist elegant microweb tec...,JVBERi0xLjQKJaqrrK0KNCAwIG9iago8PAovVGl0bGUgKE...,jalpa davedata scientist elegant microweb tech...,Jalpa Dave,0.176074,0.176927,0.000000,0.353001
